In [ ]:
import iris
from iris.analysis import AreaWeighted
from iris.analysis.trajectory import interpolate
import numpy as np
import timeit

In [ ]:
iris.__version__

In [ ]:
# from iris-test-data/test_data/NetCDF
fname = "SMALL_hires_wind_u_for_ipcc4.nc"

In [ ]:
cube = iris.load_cube(fname)[0]

In [ ]:
cube.shape

In [ ]:
cube

In [ ]:
scheme = AreaWeighted()
regridder = scheme.regridder(cube, cube)

In [ ]:
# v2.4.0
hi2lo_24 = {'1:1': 0.622, '2:1': 0.109, '3:1': 0.110, '4:1': 0.117, '5:1': 0.129, '10:1': 0.215, '20:1': 0.551}
lo2hi_24 = {'1:1': 0.622, '1:2': 1.18,  '1:3': 2.57,  '1:4': 4.68,  '1:5': 7.14,  '1:10': 29.4,  '1:20': 123.0}

# v2.3.0
hi2lo_23 = {'1:1': 8.1, '2:1': 3.45, '3:1': 3.43, '4:1':3.42, '5:1': 3.39, '10:1': 3.37, '20:1': 0}
lo2hi_23 = {'1:1': 8.1, '1:2': 13.7, '1:3': 31.5, '1:4':54.7, '1:5': 81.0, '1:10': 329.0, '1:20': 0}

In [ ]:
%%timeit -n 3 -r 3

regridder(cube)

In [ ]:
def doit(cube, factor):
    print(f'in  = {cube.shape}')
    shape = cube.shape
    plat = cube.coord('latitude').points
    plon = cube.coord('longitude').points
    sample_points = [('latitude',  np.linspace(plat[0], plon[-1], shape[0]*factor))]
    cube_2_1 = interpolate(cube, sample_points)
    sample_points = [('longitude', np.linspace(plon[0], plon[-1], shape[1]*factor))]
    cube_2_1 = interpolate(cube_2_1, sample_points)

    lat = cube_2_1.coord('latitude')
    lat = iris.coords.Coord.from_coord(lat)
    lat.bounds = None
    lat.guess_bounds()
    cube_2_1.remove_coord('latitude')
    cube_2_1.add_dim_coord(lat, 0)

    lon = cube_2_1.coord('longitude')
    lon = iris.coords.Coord.from_coord(lon)
    lon.bounds = None
    lon.guess_bounds()
    cube_2_1.remove_coord('longitude')
    cube_2_1.add_dim_coord(lon, 1)

    print(f'out = {cube_2_1.shape}')
    return cube_2_1

In [ ]:
cube_2 = doit(cube, 2)
scheme_2_1 = AreaWeighted()
regridder_2_1 = scheme_2_1.regridder(cube_2, cube)
scheme_1_2 = AreaWeighted()
regridder_1_2 = scheme_1_2.regridder(cube, cube_2)

In [ ]:
%%timeit -n 3 -r 1

regridder_2_1(cube_2)

In [ ]:
%%timeit -n 3 -r 1

regridder_1_2(cube)

In [ ]:
cube_3 = doit(cube, 3)
scheme_3_1 = AreaWeighted()
regridder_3_1 = scheme_3_1.regridder(cube_3, cube)
scheme_1_3 = AreaWeighted()
regridder_1_3 = scheme_1_3.regridder(cube, cube_3)

In [ ]:
%%timeit -n 3 -r 3

regridder_3_1(cube_3)

In [ ]:
%%timeit -n 3 -r 3

regridder_1_3(cube)

In [ ]:
cube_4 = doit(cube, 4)
scheme_4_1 = AreaWeighted()
regridder_4_1 = scheme_4_1.regridder(cube_4, cube)
scheme_1_4 = AreaWeighted()
regridder_1_4 = scheme_1_4.regridder(cube, cube_4)

In [ ]:
%%timeit -n 3 -r 3

regridder_4_1(cube_4)

In [ ]:
%%timeit -n 3 -r 1

regridder_1_4(cube)

In [ ]:
cube_5 = doit(cube, 5)
scheme_5_1 = AreaWeighted()
regridder_5_1 = scheme_5_1.regridder(cube_5, cube)
scheme_1_5 = AreaWeighted()
regridder_1_5 = scheme_1_5.regridder(cube, cube_5)

In [ ]:
%%timeit -n 3 -r 3

regridder_5_1(cube_5)

In [ ]:
%%timeit -n 3 -r 3

regridder_1_5(cube)

In [ ]:
cube_10 = doit(cube, 10)
scheme_10_1 = AreaWeighted()
regridder_10_1 = scheme_10_1.regridder(cube_10, cube)
scheme_1_10 = AreaWeighted()
regridder_1_10 = scheme_1_10.regridder(cube, cube_10)

In [ ]:
%%timeit -n 3 -r 3

regridder_10_1(cube_10)

In [ ]:
%%timeit -n 1 -r 3

regridder_1_10(cube)

In [ ]:
cube_20 = doit(cube, 20)
scheme_20_1 = AreaWeighted()
regridder_20_1 = scheme_20_1.regridder(cube_20, cube)
scheme_1_20 = AreaWeighted()
regridder_1_20 = scheme_1_20.regridder(cube, cube_20)

In [ ]:
%%timeit -n 3 -r 3

regridder_20_1(cube_20)

In [ ]:
%%timeit -n 3 -r 3

regridder_1_20(cube)

In [ ]:
hi2lo_24 = [0.622, 0.109, 0.110, 0.117, 0.129, 0.215]
hi2lo_23 = [8.1,   3.45,  3.43,  3.42,  3.39,  3.37]

lo2hi_24 = [0.622, 1.18,  2.57,  4.68,  7.14,  29.4]
lo2hi_23 = [8.1,   13.7,  31.5,  54.7,  81.0,  329.0]

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.animation as animation
from matplotlib.collections import PolyCollection
from scipy.interpolate import interp1d


class SubplotAnimation(animation.TimedAnimation):
    def __init__(self):
        self.hi2lo_24 = [0.622, 0.109, 0.110, 0.117, 0.129, 0.215]
        self.hi2lo_23 = [8.1,   3.45,  3.43,  3.42,  3.39,  3.37]
        self.lo2hi_24 = [0.622, 1.18,  2.57,  4.68,  7.14,  29.4]
        self.lo2hi_23 = [8.1,   13.7,  31.5,  54.7,  81.0,  329.0]
        
        assert len(self.hi2lo_24) == len(self.hi2lo_23) == len(self.lo2hi_24) == len(self.lo2hi_23)
        
        self.xmin, self.xmax = 0, len(self.lo2hi_24) - 1
        self.xticks = list(range(self.xmax + 1))
        
        self.f_hi2lo_24 = interp1d(self.xticks, self.hi2lo_24)
        self.f_hi2lo_23 = interp1d(self.xticks, hi2lo_23)
        self.f_lo2hi_24 = interp1d(self.xticks, lo2hi_24)
        self.f_lo2hi_23 = interp1d(self.xticks, lo2hi_23)
        
        self.xdata = []
        self.ydata_hi2lo_24 = []
        self.ydata_hi2lo_23 = []
        self.ydata_lo2hi_24 = []
        self.ydata_lo2hi_23 = []
        
        #fig = plt.figure()
        #ax1 = fig.add_subplot(1, 2, 1)
        #ax2 = fig.add_subplot(1, 2, 2)
        
        width_ratios = [1, 1]
        height_ratios = [0, 0.90, 0.10]
        gridspec_kw = dict(width_ratios=width_ratios, height_ratios=height_ratios)
        fig, axes = plt.subplots(ncols=2, nrows=3, constrained_layout=False, gridspec_kw=gridspec_kw, figsize=(8, 4))
        dummy1, (ax1, ax2), dummy2 = axes
        [ax.remove() for ax in list(dummy1) + list(dummy2)]
        
        self.lines = []
        self.sliders = [Line2D([], [], marker='o', color="r", markeredgecolor="r", markersize=6, zorder=10) for _ in range(4)]
        self.markers = [Line2D([], [], marker='s', color="k", markeredgecolor="k", markersize=4) for _ in range(24)]
        self.cursors = [Line2D([], [], lw=1, linestyle="--", color="orange"),
                        Line2D([], [], lw=1, linestyle="--", color="blue"),
                        Line2D([], [], lw=1, linestyle="--", color="orange"),
                        Line2D([], [], lw=1, linestyle="--", color="blue")]
        
        ax1.set_title("High to Low Resolution", fontsize=10)
        ax1.set_xlabel(r"src:$tgt^\dagger$ ratio")
        ax1.set_xticks(self.xticks)
        ax1.set_xticklabels(["1:1", "2:1", "3:1", "4:1", "5:1", "10:1"])
        ax1.set_yticks([0, 2, 4, 6, 8])
        ax1.set_ylabel("time / seconds")
        self.line_hi2lo_24 = Line2D([], [], lw=2, color="orange")
        self.lines.append(self.line_hi2lo_24)
        self.line_hi2lo_23 = Line2D([], [], lw=2, color="blue")
        self.lines.append(self.line_hi2lo_23)
        ax1.add_line(self.line_hi2lo_24)
        ax1.add_line(self.line_hi2lo_23)
        [ax1.add_line(marker) for marker in self.sliders[:2]]
        [ax1.add_line(marker) for marker in self.markers[:12]]
        [ax1.add_line(cursor) for cursor in self.cursors[:2]]
        ax1.set_xlim(self.xmin - 1, self.xmax + 1)
        ax1.set_ylim(-0.25, 10)
        ax1.legend([self.line_hi2lo_23, self.line_hi2lo_24],
                   ["iris 2.3.0", "iris 2.4.0"])
        ax1.grid()
        
        ax2.set_title("Low to High Resolution", fontsize=10)
        ax2.set_xlabel(r"$src^\dagger$:tgt ratio")
        ax2.set_xticks(self.xticks)
        ax2.set_xticklabels(["1:1", "1:2", "1:3", "1:4", "1:5", "1:10"])
        ax2.set_ylabel("time / seconds")
        ax2.yaxis.set_label_position("right")
        ax2.yaxis.tick_right()
        self.line_lo2hi_24 = Line2D([], [], lw=2, color="orange")
        self.lines.append(self.line_lo2hi_24)
        self.line_lo2hi_23 = Line2D([], [], lw=2, color="blue")
        self.lines.append(self.line_lo2hi_23)
        ax2.add_line(self.line_lo2hi_24)
        ax2.add_line(self.line_lo2hi_23)
        [ax2.add_line(marker) for marker in self.sliders[2:]]
        [ax2.add_line(marker) for marker in self.markers[12:]]
        [ax2.add_line(cursor) for cursor in self.cursors[2:]]
        ax2.set_xlim(self.xmin - 1, self.xmax + 1)
        ax2.set_ylim(-10, 340)
        ax2.legend([self.line_lo2hi_23, self.line_lo2hi_24],
                   ["iris 2.3.0", "iris 2.4.0"],
                   loc="upper left")
        ax2.grid()
        
        fig.suptitle("Benchmark: iris.analysis.AreaWeighted regridding scheme")
        
        txt = r"$\dagger$ "
        txt += "Using 2D Iris cube, shape (y:160, x:320)\n"
        #txt += r"$\bullet$ "
        txt += "Benchmarked on 4x Intel(R) Xeon(R) CPU E5-2690 v4 @ 2.60GHz, 6GB, RHEL 7.7 (Maipo)"
        fig.text(.5, .03, txt, ha='center', fontsize=8)
        
        self.ax1 = ax1
        self.ax2 = ax2
        
        animation.TimedAnimation.__init__(self, fig, interval=50, repeat=False, blit=True)

    def plot_markers(self, x):
        if not self.xticks and x == self.xmin:
            self.xticks = list(range(self.xmax + 1))
            
        if self.xticks and x >= self.xticks[0]:
            offset = self.xmax + 1
            xtick = self.xticks.pop(0)
            self.markers[xtick + offset*0].set_data(x, self.ydata_hi2lo_24[-1])
            self.markers[xtick + offset*1].set_data(x, self.ydata_hi2lo_23[-1])
            self.markers[xtick + offset*2].set_data(x, self.ydata_lo2hi_24[-1])
            self.markers[xtick + offset*3].set_data(x, self.ydata_lo2hi_23[-1])
        
        if x >= self.xmax:
            for marker in self.sliders:
                marker.set_visible(False)
        
    def _draw_frame(self, x):
        if x <= self.xmax:
            self.xdata.append(x)

            y = self.f_hi2lo_24(x)
            self.sliders[0].set_data(x, y)
            self.ydata_hi2lo_24.append(y)
            self.line_hi2lo_24.set_data(self.xdata, self.ydata_hi2lo_24)
        
            y = self.f_hi2lo_23(x)
            self.sliders[1].set_data(x, y)
            self.ydata_hi2lo_23.append(y)
            self.line_hi2lo_23.set_data(self.xdata, self.ydata_hi2lo_23)

            y = self.f_lo2hi_24(x)
            self.sliders[2].set_data(x, y)
            self.ydata_lo2hi_24.append(y)
            self.line_lo2hi_24.set_data(self.xdata, self.ydata_lo2hi_24)
        
            y = self.f_lo2hi_23(x)
            self.sliders[3].set_data(x, y)
            self.ydata_lo2hi_23.append(y)
            self.line_lo2hi_23.set_data(self.xdata, self.ydata_lo2hi_23)
        
        self.plot_markers(x)
        
        if x <= self.xmax:
            self.cursors[0].set_data(self.ax1.get_xlim(), [self.ydata_hi2lo_24[-1]]*2)
            self.cursors[1].set_data(self.ax1.get_xlim(), [self.ydata_hi2lo_23[-1]]*2)
            self.cursors[2].set_data(self.ax2.get_xlim(), [self.ydata_lo2hi_24[-1]]*2)
            self.cursors[3].set_data(self.ax2.get_xlim(), [self.ydata_lo2hi_23[-1]]*2)
        else:
            visible = True
            self.cursors[0].set_visible(visible)
            self.cursors[1].set_visible(visible)
            self.cursors[2].set_visible(visible)
            self.cursors[3].set_visible(visible)
        
        self._drawn_artists = self.lines + self.markers + self.sliders + self.cursors

    def new_frame_seq(self):
        return iter(np.linspace(self.xmin, self.xmax+2, 150))

    def _init_draw(self):
        for line in self.lines:
            line.set_data([], [])
        for marker in self.markers:
            marker.set_data([], [])
        for marker in self.sliders:
            marker.set_data([], [])
        for cursor in self.cursors:
            cursor.set_data([], [])

        
ani = SubplotAnimation()
#ani.save("twitter.mp4")